# P2 - permutation-equiriant functions

In [6]:
# data preprocessing modules
from sklearn import metrics

# ML modules
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Input, Activation, MaxPooling1D, Conv1D, Flatten
from keras.optimizers import Adam
from keras.utils import plot_model

# general stuff
import pandas as pd
import io
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### P2.1

In [2]:
X_train = pd.read_csv('./data/xtrain-2.csv').drop(['Unnamed: 0'], axis=1) # Removes one column to clean data
y_train = pd.read_csv('./data/ytrain-2.csv').drop(['Unnamed: 0'], axis=1)
X_test = pd.read_csv('./data/xtest-2.csv').drop(['Unnamed: 0'], axis=1)
y_test = pd.read_csv('./data/ytest-2.csv').drop(['Unnamed: 0'], axis=1)

print(f'X_train {X_train.shape}')

# Reshape X_train to be right dimentions for conv1D layer (below)
X_train_3d = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_3d = X_test.values.reshape(X_test.shape[0], X_train.shape[1], 1)
print(f'X_train_3d {X_train_3d.shape}')

X_train (200000, 15)
X_train_3d (200000, 15, 1)


In [16]:
# Define input features
n_features = X_train_3d.shape[1:]
n_features

(15, 1)

In [17]:
# Construct permutation equivariant neural network
def create_p_e_model(L, w):
  inputs = Input(shape=(X_train_3d.shape[1], X_train_3d.shape[2]))
  print(inputs.shape)
  x = inputs
  for i in range(L-1):
    x = Conv1D(5, w, padding='same', activation='relu', name='conv_layer_{}'.format(i))(x)
    x = Activation('relu')(x)
  
  x = MaxPooling1D(pool_size=x.shape[1])(x)
  flatten = Flatten()(x)
  outputs = Dense(5)(flatten)

  p_e_model = keras.Model(inputs = inputs, outputs=outputs)
  
  optimizer = Adam(learning_rate=1e-4, epsilon=1e-3)
  p_e_model.compile(optimizer=optimizer, loss='mse', metrics=['accuracy'])
  
  return p_e_model

In [18]:
p_e_model = create_p_e_model(L=10,w=5)
p_e_model.summary()

(None, 15, 1)
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 15, 1)]           0         
                                                                 
 conv_layer_0 (Conv1D)       (None, 15, 5)             30        
                                                                 
 activation_27 (Activation)  (None, 15, 5)             0         
                                                                 
 conv_layer_1 (Conv1D)       (None, 15, 5)             130       
                                                                 
 activation_28 (Activation)  (None, 15, 5)             0         
                                                                 
 conv_layer_2 (Conv1D)       (None, 15, 5)             130       
                                                                 
 activation_29 (Activation)  (None, 15, 5)   

In [19]:
# code for generating attached image
# plot_model(p_e_model, 'p_e_model_model.png', show_shapes=True)

Explanation of implementation:
- This model uses the Functional API from Keras. I chose that, not the sequential as it provides more flexibility.
- the function accepts two parameters (L, w), which gets pased down. L being number of layers and w being inserted to the first conv1D layer as the `kernel_size`. By having the `filter`to 5 (m) it creates `mxw` dimention.
- For the L-1 layers there is a consists of an equivariant affine transformation followed by ReLU activation layer. 
- Conv1D makes sure the network is eqivariant as it independantly applyes filters to each feature channel. In that way the network learns features that are equivarant to transformations within each channel.
- Then there is a maxPooling layer that that takes the maximum value over each feature channel of the output of the last convolutional layer. This ensures to create a translation-equivariant neural network that is invariant to the ordering of the input sequence.
- It is then flattened and passed to the output layer with 5 (m) dimentions.

### P2.2

In [20]:
L = [2,2,2,2,3,3,3,3]
w = [5,10,100,200,5,10,100,200]

zipped = zip(L,w) # zips and creates tuples of each L and w value

hist_acc_p_e = []
hist_val_acc_p_e = []
hist_loss_p_e = []
hist_val_loss_p_e = []

val_split = 0.1
n_epochs = 100
batch_size = 300

for L_val,w_val in zip(L,w):
  
  print("Values: (%s, %s)" %(int(L_val), int(w_val)))
  print("-----------------------------\n")

  model = create_p_e_model(L_val,w_val)

  history_p_e = model.fit(X_train_3d,                          
                  y_train,                          
                  epochs=n_epochs,                  
                  batch_size=batch_size,  
                  validation_split=val_split
                  )
  pred = model.predict(X_test_3d)
  score = np.sqrt(metrics.mean_squared_error(pred, y_test))
  print(f"--- Final score (RMSE) : {score}")
  
  # Store the performance
  hist_acc_p_e.append(history_p_e.history['accuracy'])
  # hist_val_acc_p_e.append(history_p_e.history['val_acc'])
  hist_loss_p_e.append(history_p_e.history['loss'])
  hist_val_loss_p_e.append(history_p_e.history['val_loss'])

  print("-----------------------------\n")

Values: (2, 5)
-----------------------------

(None, 15, 1)
Epoch 1/100


2023-02-24 15:57:11.891043: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-24 15:57:12.365149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


595/600 [============================>.] - ETA: 0s - loss: 0.7503 - accuracy: 0.2002

2023-02-24 15:57:17.885014: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


600/600 [==============================] - 6s 8ms/step - loss: 0.7481 - accuracy: 0.2003 - val_loss: 0.5011 - val_accuracy: 0.2017
Epoch 2/100
600/600 [==============================] - 4s 7ms/step - loss: 0.3715 - accuracy: 0.1999 - val_loss: 0.2797 - val_accuracy: 0.2033
Epoch 3/100
600/600 [==============================] - 4s 7ms/step - loss: 0.2353 - accuracy: 0.1998 - val_loss: 0.2064 - val_accuracy: 0.2029
Epoch 4/100
600/600 [==============================] - 4s 7ms/step - loss: 0.1939 - accuracy: 0.2009 - val_loss: 0.1861 - val_accuracy: 0.2000
Epoch 5/100
600/600 [==============================] - 4s 7ms/step - loss: 0.1818 - accuracy: 0.2008 - val_loss: 0.1787 - val_accuracy: 0.2025
Epoch 6/100
600/600 [==============================] - 4s 7ms/step - loss: 0.1760 - accuracy: 0.2010 - val_loss: 0.1740 - val_accuracy: 0.2018
Epoch 7/100
600/600 [==============================] - 4s 7ms/step - loss: 0.1720 - accuracy: 0.2015 - val_loss: 0.1706 - val_accuracy: 0.2024
Epoch 8/100

...

6250/6250 [==============================] - 11s 2ms/step
--- Final score (RMSE) : 0.3999953687227674
-----------------------------

Values: (3, 10)
-----------------------------

(None, 15, 1)
Epoch 1/100
  1/600 [..............................] - ETA: 3:10 - loss: 0.3216 - accuracy: 0.2467

2023-02-24 16:35:12.737978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


598/600 [============================>.] - ETA: 0s - loss: 0.2045 - accuracy: 0.1993

2023-02-24 16:35:17.264089: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


600/600 [==============================] - 5s 8ms/step - loss: 0.2044 - accuracy: 0.1993 - val_loss: 0.1728 - val_accuracy: 0.2045
Epoch 2/100
600/600 [==============================] - 4s 7ms/step - loss: 0.1704 - accuracy: 0.2005 - val_loss: 0.1682 - val_accuracy: 0.2068
Epoch 3/100
600/600 [==============================] - 4s 7ms/step - loss: 0.1671 - accuracy: 0.2005 - val_loss: 0.1658 - val_accuracy: 0.2058
Epoch 4/100
600/600 [==============================] - 5s 8ms/step - loss: 0.1651 - accuracy: 0.2014 - val_loss: 0.1642 - val_accuracy: 0.2056
Epoch 5/100
600/600 [==============================] - 4s 7ms/step - loss: 0.1637 - accuracy: 0.2014 - val_loss: 0.1631 - val_accuracy: 0.2033
Epoch 6/100
600/600 [==============================] - 5s 8ms/step - loss: 0.1628 - accuracy: 0.2007 - val_loss: 0.1623 - val_accuracy: 0.2037
Epoch 7/100
600/600 [==============================] - 5s 8ms/step - loss: 0.1621 - accuracy: 0.2005 - val_loss: 0.1618 - val_accuracy: 0.2045
Epoch 8/100

In [ ]:
colors =['b','g','r','c','m','y','k'] # lists of color to each tuple

for i, (L_val, w_val) in enumerate(zip(L, w)):
    label = f"({L_val}, {w_val})"
    color = colors[i % len(colors)] # select a color based on the index
    plt.plot(hist_loss_p_e[i], '-|', label=label, color=color)
    plt.plot(hist_val_loss_p_e[i], '-x', label=label, color=color)
    
plt.title('Model Loss on Training data')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.figure(figsize=(15,15))
plt.show()

Comments:
- It is hard to accuratly distinguish betweeen the train/val for each one, but the figure is good in order to see the comparison between the sets. However, the best performing (3,200) is the darkblue and we can see that the validation (x) started higher than the loss (|) and then they converge. This indicates that the more complex set of L and w gives the best performance with the current architecture. However, on 100 epochs. All converged.